# CRUD API
## Ingesting video from external source

Calling ingest:

1. Check if there is already an object with this URL in kv storage? If there is, just return metadata stored there
2. If there is none, download the video locally, depending on the protocol (youtube - ok, youtube, if it is telegram video then telegram etc).
3. Then save it to bucket
4. And finally, store the serialized `Media` into the doc storage. Both video and audio URL should point to the same file, with relevant metadata.

Thus, we have:
1. Natural caching. Question here, are we OK with immutable URLs? I.e. can we judge that data behind the URL should never change? this is the case with YT and Telegram, but this sounds more general.
2. Metadata is cached too. We Saving metadata should be transactional with saving data to the bucket. If we need to check that meta only no need to download file.
3. Source to interface adaptor. Once ingested (via simple `str` parameter of URL) all the video is treated same.

Keys in cache would be of two kinds - external URLs for externally-originated resources, and internal `gs://` for internal resources (dubs etc)

In [1]:
import aiohttp

from freespeech import client
from freespeech.types import Job, Media

async with aiohttp.ClientSession() as session:

    # this ingests video from youtube.
    download_job:Job[Media] = await client.ingest_video(url="https://youtube.com/?v=ASDLKJFH", session=session)

    # this hits cache and thus just returns meta from cache
    download_job = await client.ingest_video(url="https://youtube.com/?v=ASDLKJFH", session=session)

    # this downloads video from telegram
    download_job = await client.ingest_video(url="telegram://video/IDENTIFIER", session=session)

InvalidURL: /upload

## Getting metadata

To get a local copy for further processing, call `get_video(url:str)`. Within get_video we can do following:

1. Check if there is a key in metadata storage, if there is none - try to `ingest`.
2. Return metadata

To get actual file if needed for processing, still should use `objectstorage.get`

In [2]:
    # this has already been ingested
    result = await client.media(session= session, url="https://youtube.com/?v=ASDLKJFH")

    # something which is not in system. Lack of this should mean application error, we should well control when we should have media for sure and when not.
    result = await client.media(session= session, url="https://youtube.com/?v=NONEXIST") # error

RuntimeError: Session is closed

## Saving clip

We should have a method somewhere to save local stream (file) to bucket + metadata. It should only be one entry point, so that metadata is stored on write always. Something like
`upload_media(local_file:Path)` which would calculate the meta and put it in the right place in obj storage.

We are doing it now in dub.py, maybe just extracting to a method be enough?